# Ethereum与BitCoin的区别
## 1. 账户包含的内容（热钱包）
### 比特币：
- UTXO，即对于以前交易输出的引用信息。
- 公钥。

### 以太坊
- 随机数:用于确定每笔交易只能被处理一次
- 账户目前的以太币余额
- 账户的合约代码
- 账户的存储

## 2. 账户的类型
### 比特币：
- 只有一种，即钱包账户（由私钥控制）

### 以太坊：
- 外部所有的账户（由私钥控制）

<em>人们通过创建和签名一笔交易从外部账户发送消息</em>
- 合约账户（由合约代码控制）

<em>合约账户收到消息，账户内部的代码就被激活，允许对内部存储进行读取和写入，发送其它消息或创建合约</em>

## 3. 交易的过程
### a) 比特币交易&以太坊消息
- 比特币交易只能发送交易数量及身份认证
- 以太坊消息可以发送包含数据的消息

### b) 以太坊的消息可以由外部账户或合约账户发起
### c) 如果以太坊消息的接收者是合约账户，可以选择回应

# 以太坊的交易过程
<em>以太坊的交易是指存储从外部账户发出的消息的签名数据包</em>

## 交易包含的内容
- 交易的接收者
- 用于确认发送者的签名
- 以太币账户余额
- 要发送的数据
- STARTGAS&GASPRICE

<em>交易过程中若GAS不够用了即会回退交易，若交易完成且GAS有剩余则会将GAS返还</em>

## 状态转换函数
以太坊的状态转换函数： $ APPLY(S,TX) -> S' $

1. 检查交易的格式是否正确、签名是否有效和<font color=#FF0033>随机数是否与发送者账户的随机数匹配</font>。
2. 计算交易费用 $ fee=STARTGAS * GASPRICE $，并从签名中确认发送者的地址。<em>之后从发送者账户中减去交易费用并<font color=#FF0033>增加发送者的随机数</font></em>
3. 设定交易的初始$ GAS = STARTGAS $，之后按照交易中的字节数减去一定量的GAS
4. 从发送者账户转移价值到接收者账户，若账户不存在则新建。如果接收账户是一个合约账户，则运行合约的代码，直到代码运行结束或GAS用尽。
5. GAS用尽则交易失败回滚，GAS不返还。交易完成则返还剩下的GAS

## 以太坊代码执行
<em>以太坊合约的代码使用低级的基于堆栈的字节码语言写成（Ethereum虚拟代码/EVM代码）</em>

<em>以太坊的代码执行是无限循环，直到遇到STOP或RETURN指令停止。</em>
### 以太坊可以访问的三种存储数据方式
- 堆栈，32字节的数据可以入栈出栈
- 内存，可无限扩展的字节队列
- 合约的长期存储，一个Key:Value的存储形式，Key/Value都是32字节大小，存储内容长期保持。

### 区块确认算法
<em> 安全验证 </em>
1. 检查区块引用的上一个区块是否<font color=#FF0033>存在和有效</font>
2. 检查区块的<font color=#FF0033>时间戳</font>是否比上一个区块大且小于15分钟（即此事件是否在前一个区块之前发生）
3. 检查区块序号、难度值、交易根，叔根和瓦斯限额是否有效
4. 检查工作量证明是否有效（哈希合法）

<em>状态转换</em>
5. 将$ S[0] $赋值为上一个区块的状态<font color=#FF0033>STATE_ROOT</font>
6. 将TX赋值为区块的交易列表，一共有n笔交易，对于属于0...n-1的i,进行状态转换$ S[i+1] = APPLY(S[i],TX[i]) $，中途发生错误或GAS耗尽，则返回错误。
7. 用$ S[n] $给$ S\_FINAL $赋值，向矿工发放奖励。
8. 检查$ S\_FINAL $是否与<font color=#FF0033>STATE_ROOT</font>相同，若相同则有效。


### 区块状态的存储方式（Merkle Patrica Tree:梅克尔帕特里夏树）

#### MPT是Merkle树与前缀树的融合

状态存储在树结构中（tree structure），每增加一个区块只需要改变树结构的一小部分。因此，一般而言，<font color=#00ACC1>两个相邻的区块的树结构的大部分应该是相同的</font>，因此存储一次数据，可以利用指针（即子树哈希）引用两次。一种被称为“帕特里夏树”（“Patricia Tree”）的树结构可以实现这一点，其中包括了对默克尔树概念的修改，不仅允许改变节点，而且还可以插入和删除节点。另外，<font color=#FF0033>因为所有的状态信息是最后一个区块的一部分，所以没有必要存储全部的区块历史</font>-这一方法如果能够可以应用到比特币系统中，经计算可以对存储空间有10-20倍的节省。